# High-level Comparison Between the Network Covariance and Characteristic Matrices

In this notebook, we provide a rough comparison between covariance-based and entropy-based topology inference methods.
The goal is to test each method across a range of entangled states to get an idea of their relative performance.


## Key Differences

### Covariance Matrix Advantages

There is a significant performance advantage over the characteristic matrix:
* This can easily be observed by noting the cell execution times, especially when more qubits are considered. 
* The covariance is evaluated from one circuit evaluation $O(1)$, whereas the charcteristic matrix requires a minimum of $O(n^2)$ circuit evaluations where $n$ is the number of qubits. Given clever parallelization in quantum circuits, the characteristic matrix may be evaluated in $O(n)$.
* Optimizing the covariance matrix seems to require significantly fewer steps than needed to successfully infer topology using the characteristic matrix. This means that fewer circuits need to be run.
* The optimization appears to be more robust and less susceptible to local minima. We see this in the fact that the characteristic matrix often fails to infer the GHZ states with more than 3 qubits.


### Characteristic Matrix Advantages

A key difference between the covariance and characteristic matrices is that the covariance matrix seems to always have ones along the diagonal, even if the measured qubit is not correlated with any other qubits. On the contrary, the characteristic matrix measures a zero von neumann entropy if there is no correlation whatsoever.

The threshold of deciding if a correlation exists needs to be raised as the number of shots decreases.
The rate at which covariance methods need to be raised seems to more than characteristic matrices.
This may mean, that the variance of the entropic quantities in the low shot regime are improved over those of the covariance matrix.

In [1]:
import qnetti
import pennylane as qml
from pennylane import numpy as qnp
import qnetvo

print(qnetvo.__version__)
print(qml.__version__)

0.4.1
0.28.0


In [2]:
def decision_matrix(mat, atol=0.1):
    return qnp.where(qnp.abs(mat) > atol, 1, 0)


def matrix_distance(mat1, mat2):
    return qnp.linalg.norm(qnp.array(mat1) - qnp.abs(qnp.array(mat2)))


def characteristic_matrix_inference(prep_node, expected_mat, **kwargs):
    char_mat, mi_opt_dict, vn_opt_dict = qnetti.optimize_characteristic_matrix(prep_node, **kwargs)
    dec_mat = decision_matrix(char_mat)
    char_dist = matrix_distance(expected_mat, char_mat)

    print("characteristic matrix :\n", char_mat)
    print("decision matrix :\n", dec_mat)
    print("expected matrix :\n", expected_mat)
    print("distance to expected : ", char_dist)


def covariance_matrix_inference(
    prep_node,
    expected_mat,
    **kwargs,
):
    # settings = qnp.random.rand(3 * num_qubits, requires_grad=True)
    # cov_cost = qnetti.qubit_covariance_cost_fn(
    #     prep_node, meas_wires=meas_wires, shots=shots, qnode_kwargs=qnode_kwargs
    # )

    # settings_list = [settings]
    # cost_vals = [cov_cost(settings)]

    # opt = qml.GradientDescentOptimizer(stepsize=step_size)
    # for i in range(num_steps):
    #     settings, cost_val = opt.step_and_cost(cov_cost, settings)

    #     cost_vals += [cost_val]
    #     settings_list += [settings]

    # min_id = qnp.argmin(cost_vals)
    # min_cost = cost_vals[min_id]
    # opt_settings = settings_list[min_id]

    # cov_mat = qnetti.qubit_covariance_matrix_fn(
    #     prep_node, meas_wires=meas_wires, shots=shots, qnode_kwargs=qnode_kwargs
    # )(opt_settings)
    # dec_mat = decision_matrix(cov_mat, atol=atol)
    # cov_dist = matrix_distance(expected_mat, cov_mat)

    cov_mat, opt_dict = qnetti.optimize_covariance_matrix(prep_node, **kwargs)
    dec_mat = decision_matrix(cov_mat)

    print("covariance matrix :\n", cov_mat)
    print("decision matrix :\n", dec_mat)
    print("expected matrix :\n", expected_mat)

## Two-Qubit GHZ State

In [3]:
ghz2_mat = qnp.ones((2, 2))


def rot_bell_state(settings, wires):
    qnetvo.ghz_state(settings, wires)
    qml.ArbitraryUnitary([qnp.pi / 4, 0.3, -1.5], wires=[wires[0]])
    qml.ArbitraryUnitary([-0.7, 2.9, 1.2], wires=[wires[1]])


bell_state_prep_node = qnetvo.PrepareNode(wires=[0, 1], ansatz_fn=rot_bell_state)

In [4]:
characteristic_matrix_inference(
    bell_state_prep_node, ghz2_mat, mi_opt_kwargs={"num_steps": 15, "step_size": 0.1}
)

characteristic matrix :
 [[1.         0.99997966]
 [0.99997966 1.        ]]
decision matrix :
 [[1 1]
 [1 1]]
expected matrix :
 [[1. 1.]
 [1. 1.]]
distance to expected :  2.8771619012054495e-05


In [5]:
covariance_matrix_inference(
    bell_state_prep_node,
    ghz2_mat,
    step_size=0.2,
    num_steps=6,
    verbose=False,
)

covariance matrix :
 [[1.         0.99999918]
 [0.99999918 1.        ]]
decision matrix :
 [[1 1]
 [1 1]]
expected matrix :
 [[1. 1.]
 [1. 1.]]


## 3-Qubit GHZ state

In [6]:
ghz3_mat = qnp.ones((3, 3))


def rot_ghz3_state(settings, wires):
    qnetvo.ghz_state(settings, wires)
    qml.ArbitraryUnitary([qnp.pi / 4, 0.3, -1.5], wires=[wires[0]])
    qml.ArbitraryUnitary([-0.7, 2.9, 1.2], wires=[wires[1]])
    qml.ArbitraryUnitary([-0.9, 0.9, 0.12], wires=[wires[2]])


ghz3_state_prep_node = qnetvo.PrepareNode(wires=[0, 1, 2], ansatz_fn=rot_ghz3_state)

In [11]:
characteristic_matrix_inference(
    ghz3_state_prep_node,
    ghz3_mat,
    mi_opt_kwargs={"num_steps": 40, "step_size": 0.4},
    use_measured_mutual_info=False,
)

characteristic matrix :
 [[1.         0.98950007 0.99026563]
 [0.98950007 1.         0.98767853]
 [0.99026563 0.98767853 1.        ]]
decision matrix :
 [[1 1 1]
 [1 1 1]
 [1 1 1]]
expected matrix :
 [[1. 1. 1.]
 [1. 1. 1.]
 [1. 1. 1.]]
distance to expected :  0.026714230804181914


In [12]:
characteristic_matrix_inference(
    ghz3_state_prep_node,
    ghz3_mat,
    mi_opt_kwargs={"num_steps": 40, "step_size": 0.4},
    use_measured_mutual_info=True,
)

characteristic matrix :
 [[1.         0.98903036 0.99691531]
 [0.98903036 1.         0.98888036]
 [0.99691531 0.98888036 1.        ]]
decision matrix :
 [[1 1 1]
 [1 1 1]
 [1 1 1]]
expected matrix :
 [[1. 1. 1.]
 [1. 1. 1.]
 [1. 1. 1.]]
distance to expected :  0.022516419587203233


In [13]:
covariance_matrix_inference(
    ghz3_state_prep_node,
    ghz3_mat,
    step_size=0.1,
    num_steps=15,
    verbose=False,
)

covariance matrix :
 [[ 1.          0.99999998 -0.99999998]
 [ 0.99999998  1.         -1.        ]
 [-0.99999998 -1.          1.        ]]
decision matrix :
 [[1 1 1]
 [1 1 1]
 [1 1 1]]
expected matrix :
 [[1. 1. 1.]
 [1. 1. 1.]
 [1. 1. 1.]]


## 4-Qubit GHZ

In [14]:
ghz4_mat = qnp.ones((4, 4))


def rot_ghz4_state(settings, wires):
    qnetvo.ghz_state(settings, wires)
    qml.ArbitraryUnitary([qnp.pi / 4, 0.3, -1.5], wires=[wires[0]])
    qml.ArbitraryUnitary([-0.7, 2.9, 1.2], wires=[wires[1]])
    qml.ArbitraryUnitary([-0.9, 0.9, 0.12], wires=[wires[2]])
    qml.ArbitraryUnitary([2, 2, 2], wires=[wires[3]])


ghz4_state_prep_node = qnetvo.PrepareNode(wires=[0, 1, 2, 3], ansatz_fn=rot_ghz4_state)

In [16]:
characteristic_matrix_inference(
    ghz4_state_prep_node,
    ghz4_mat,
    mi_opt_kwargs={
        "num_steps": 30,
        "step_size": 0.1,
    },
)

characteristic matrix :
 [[1.         0.99997516 0.9999832  0.96370601]
 [0.99997516 1.         0.99996854 0.96370011]
 [0.9999832  0.99996854 1.         0.96370331]
 [0.96370601 0.96370011 0.96370331 1.        ]]
decision matrix :
 [[1 1 1 1]
 [1 1 1 1]
 [1 1 1 1]
 [1 1 1 1]]
expected matrix :
 [[1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]]
distance to expected :  0.08890880260527317


In [19]:
characteristic_matrix_inference(
    ghz4_state_prep_node,
    ghz4_mat,
    mi_opt_kwargs={
        "num_steps": 30,
        "step_size": 0.1,
    },
    use_measured_mutual_info=True,
)

characteristic matrix :
 [[1.00000000e+00 5.96948915e-01 9.99999995e-01 9.40381800e-01]
 [5.96948915e-01 1.00000000e+00 6.88401835e-01 9.36606189e-01]
 [9.99999995e-01 6.88401835e-01 1.00000000e+00 6.35817811e-04]
 [9.40381800e-01 9.36606189e-01 6.35817811e-04 1.00000000e+00]]
decision matrix :
 [[1 1 1 1]
 [1 1 1 1]
 [1 1 1 0]
 [1 1 0 1]]
expected matrix :
 [[1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]]
distance to expected :  1.591128823602618


In [21]:
covariance_matrix_inference(
    ghz4_state_prep_node,
    ghz4_mat,
    step_size=0.2,
    num_steps=6,
    verbose=False,
)

covariance matrix :
 [[ 1.          0.92092619 -0.91140803 -0.88328875]
 [ 0.92092619  1.         -0.97772161 -0.94755639]
 [-0.91140803 -0.97772161  1.          0.937763  ]
 [-0.88328875 -0.94755639  0.937763    1.        ]]
decision matrix :
 [[1 1 1 1]
 [1 1 1 1]
 [1 1 1 1]
 [1 1 1 1]]
expected matrix :
 [[1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]]


# 5-Qubit GHZ

In [22]:
ghz5_mat = qnp.ones((5, 5))


def rot_ghz5_state(settings, wires):
    qnetvo.ghz_state(settings, wires)
    qml.ArbitraryUnitary([qnp.pi / 4, 0.3, -1.5], wires=[wires[0]])
    qml.ArbitraryUnitary([-0.7, 2.9, 1.2], wires=[wires[1]])
    qml.ArbitraryUnitary([-0.9, 0.9, 0.12], wires=[wires[2]])
    qml.ArbitraryUnitary([2, 2, 2], wires=[wires[3]])
    qml.ArbitraryUnitary([1, 2.3, 0.4], wires=[wires[4]])


ghz5_state_prep_node = qnetvo.PrepareNode(wires=[0, 1, 2, 3, 4], ansatz_fn=rot_ghz5_state)

In [26]:
characteristic_matrix_inference(
    ghz5_state_prep_node,
    ghz5_mat,
    mi_opt_kwargs={"num_steps": 30, "step_size": 0.1},
)

characteristic matrix :
 [[1.         0.99359056 1.         0.97498664 0.9940281 ]
 [0.99359056 1.         0.99359056 0.9705925  0.98862049]
 [1.         0.99359056 1.         0.97498664 0.9940281 ]
 [0.97498664 0.9705925  0.97498664 1.         0.97092549]
 [0.9940281  0.98862049 0.9940281  0.97092549 1.        ]]
decision matrix :
 [[1 1 1 1 1]
 [1 1 1 1 1]
 [1 1 1 1 1]
 [1 1 1 1 1]
 [1 1 1 1 1]]
expected matrix :
 [[1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]]
distance to expected :  0.08055367245031207


In [27]:
characteristic_matrix_inference(
    ghz5_state_prep_node,
    ghz5_mat,
    mi_opt_kwargs={"num_steps": 30, "step_size": 0.1},
    use_measured_mutual_info=True,
)

characteristic matrix :
 [[1.         0.15317429 1.         0.71774698 1.        ]
 [0.15317429 1.         1.         0.1161138  0.97933034]
 [1.         1.         1.         0.98942237 0.99999999]
 [0.71774698 0.1161138  0.98942237 1.         0.47251307]
 [1.         0.97933034 0.99999999 0.47251307 1.        ]]
decision matrix :
 [[1 1 1 1 1]
 [1 1 1 1 1]
 [1 1 1 1 1]
 [1 1 1 1 1]
 [1 1 1 1 1]]
expected matrix :
 [[1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]]
distance to expected :  1.9270791087042223


In [28]:
covariance_matrix_inference(
    ghz5_state_prep_node, ghz5_mat, step_size=0.1, num_steps=10, verbose=False
)

covariance matrix :
 [[ 1.          0.99999991 -0.99999999 -0.98622572  0.99999822]
 [ 0.99999991  1.         -0.99999992 -0.98622565  0.99999814]
 [-0.99999999 -0.99999992  1.          0.98622573 -0.99999822]
 [-0.98622572 -0.98622565  0.98622573  1.         -0.98622398]
 [ 0.99999822  0.99999814 -0.99999822 -0.98622398  1.        ]]
decision matrix :
 [[1 1 1 1 1]
 [1 1 1 1 1]
 [1 1 1 1 1]
 [1 1 1 1 1]
 [1 1 1 1 1]]
expected matrix :
 [[1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]]


# 8-qubit GHZ state

In [29]:
ghz8_mat = qnp.ones((8, 8))


def rot_ghz8_state(settings, wires):
    qnetvo.ghz_state(settings, wires)
    qml.ArbitraryUnitary([qnp.pi / 4, 0.3, -1.5], wires=[wires[0]])
    qml.ArbitraryUnitary([-0.7, 2.9, 1.2], wires=[wires[1]])
    qml.ArbitraryUnitary([-0.9, 0.9, 0.12], wires=[wires[2]])
    qml.ArbitraryUnitary([2, 2, 2], wires=[wires[3]])
    qml.ArbitraryUnitary([1, 2.3, 0.4], wires=[wires[4]])
    qml.ArbitraryUnitary([-0.9, 0.9, 0.12], wires=[wires[5]])
    qml.ArbitraryUnitary([2, 2, 2], wires=[wires[6]])
    qml.ArbitraryUnitary([1, 2.3, 0.4], wires=[wires[7]])


ghz8_state_prep_node = qnetvo.PrepareNode(
    wires=[0, 1, 2, 3, 4, 5, 6, 7],
    ansatz_fn=rot_ghz8_state,
)

In [30]:
characteristic_matrix_inference(
    ghz8_state_prep_node,
    ghz8_mat,
    mi_opt_kwargs={"num_steps": 30, "step_size": 0.1},
)

characteristic matrix :
 [[1.         0.99651186 0.98992905 0.91718827 0.99196228 0.98743777
  0.93136837 0.99547558]
 [0.99651186 1.         0.99137212 0.91809538 0.99346344 0.98882488
  0.93232561 0.997134  ]
 [0.98992905 0.99137212 1.         0.91419829 0.98725091 0.98300417
  0.92821704 0.99045558]
 [0.91718827 0.91809538 0.91419829 1.         0.91545699 0.9126064
  0.87021568 0.91752152]
 [0.99196228 0.99346344 0.98725091 0.91545699 1.         0.9848477
  0.92954295 0.99250882]
 [0.98743777 0.98882488 0.98300417 0.9126064  0.9848477  1.
  0.92654158 0.9879447 ]
 [0.93136837 0.93232561 0.92821704 0.87021568 0.92954295 0.92654158
  1.         0.93171996]
 [0.99547558 0.997134   0.99045558 0.91752152 0.99250882 0.9879447
  0.93171996 1.        ]]
decision matrix :
 [[1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1]]
expected matrix :
 [[1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 

In [32]:
characteristic_matrix_inference(
    ghz8_state_prep_node,
    ghz8_mat,
    mi_opt_kwargs={"num_steps": 30, "step_size": 0.1},
    use_measured_mutual_info=True,
)

characteristic matrix :
 [[1.00000000e+00 5.56807967e-03 7.28292798e-03 9.94677639e-01
  9.32645707e-01 1.46698830e-02 4.32586877e-02 9.99821301e-01]
 [5.56807967e-03 1.00000000e+00 2.69601911e-01 1.46464549e-04
  1.86266512e-01 9.99999998e-01 1.12158842e-02 4.09249013e-04]
 [7.28292798e-03 2.69601911e-01 1.00000000e+00 9.43272014e-01
  7.49201963e-01 9.99981254e-01 9.34008060e-01 6.34812047e-02]
 [9.94677639e-01 1.46464549e-04 9.43272014e-01 1.00000000e+00
  6.16061702e-01 9.64783739e-01 9.40512137e-01 9.33245144e-01]
 [9.32645707e-01 1.86266512e-01 7.49201963e-01 6.16061702e-01
  1.00000000e+00 9.99999998e-01 9.33939214e-01 9.99252625e-01]
 [1.46698830e-02 9.99999998e-01 9.99981254e-01 9.64783739e-01
  9.99999998e-01 1.00000000e+00 5.46480445e-01 1.17525280e-01]
 [4.32586877e-02 1.12158842e-02 9.34008060e-01 9.40512137e-01
  9.33939214e-01 5.46480445e-01 1.00000000e+00 9.62028398e-01]
 [9.99821301e-01 4.09249013e-04 6.34812047e-02 9.33245144e-01
  9.99252625e-01 1.17525280e-01 9.6202

In [34]:
covariance_matrix_inference(
    ghz8_state_prep_node,
    ghz8_mat,
    step_size=0.1,
    num_steps=10,
    verbose=False,
)

covariance matrix :
 [[ 1.          0.95279916 -0.93596336  0.94349142 -0.8701739   0.91189771
  -0.91555223 -0.95457646]
 [ 0.95279916  1.         -0.97788071  0.98574591 -0.90914485  0.95273727
  -0.95655546 -0.9973274 ]
 [-0.93596336 -0.97788071  1.         -0.96832795  0.89308042 -0.93590257
   0.93965329  0.9797048 ]
 [ 0.94349142  0.98574591 -0.96832795  1.         -0.90026356  0.94343014
  -0.94721103 -0.98758467]
 [-0.8701739  -0.90914485  0.89308042 -0.90026356  1.         -0.87011738
   0.87360447  0.91084072]
 [ 0.91189771  0.95273727 -0.93590257  0.94343014 -0.87011738  1.
  -0.91549277 -0.95451446]
 [-0.91555223 -0.95655546  0.93965329 -0.94721103  0.87360447 -0.91549277
   1.          0.95833977]
 [-0.95457646 -0.9973274   0.9797048  -0.98758467  0.91084072 -0.95451446
   0.95833977  1.        ]]
decision matrix :
 [[1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1]]
expect

# 8 qubit graph state

In [35]:
# This expected matrix is not known with certainty, but agrees with results
graph8_mat = qnp.array(
    [
        [1, 0, 0, 1, 0, 0, 0, 0],
        [0, 1, 1, 0, 0, 0, 0, 0],
        [0, 1, 1, 0, 0, 0, 0, 0],
        [1, 0, 0, 1, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 0, 0, 0],
        [0, 0, 0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 0, 0, 1, 1],
        [0, 0, 0, 0, 0, 0, 1, 1],
    ]
)

graph8_state_prep_node = qnetvo.PrepareNode(
    wires=[0, 1, 2, 3, 4, 5, 6, 7],
    ansatz_fn=qnetvo.graph_state_fn([[0, 1], [1, 2], [0, 3], [6, 7]]),
)

In [36]:
characteristic_matrix_inference(
    graph8_state_prep_node,
    graph8_mat,
    mi_opt_kwargs={
        "num_steps": 60,
        "step_size": 0.02,
    },
)

characteristic matrix :
 [[1.00000000e+00 3.99680289e-15 4.21884749e-15 7.54810721e-01
  3.99680289e-15 4.21884749e-15 4.21884749e-15 3.99680289e-15]
 [3.99680289e-15 1.00000000e+00 1.27712390e-04 4.44089210e-15
  4.21884749e-15 3.99680289e-15 4.44089210e-15 3.99680289e-15]
 [4.21884749e-15 1.27712390e-04 1.00000000e+00 4.44089210e-15
  3.99680289e-15 3.99680289e-15 4.21884749e-15 4.44089210e-15]
 [7.54810721e-01 4.44089210e-15 4.44089210e-15 1.00000000e+00
  3.77475828e-15 3.99680289e-15 3.77475828e-15 3.99680289e-15]
 [3.99680289e-15 4.21884749e-15 3.99680289e-15 3.77475828e-15
  2.99945761e-09 3.99680289e-15 4.21884749e-15 4.21884749e-15]
 [4.21884749e-15 3.99680289e-15 3.99680289e-15 3.99680289e-15
  3.99680289e-15 3.05610876e-09 3.99680289e-15 4.21884749e-15]
 [4.21884749e-15 4.44089210e-15 4.21884749e-15 3.77475828e-15
  4.21884749e-15 3.99680289e-15 1.00000000e+00 9.92337293e-01]
 [3.99680289e-15 3.99680289e-15 4.44089210e-15 3.99680289e-15
  4.21884749e-15 4.21884749e-15 9.9233

In [37]:
characteristic_matrix_inference(
    graph8_state_prep_node,
    graph8_mat,
    mi_opt_kwargs={
        "num_steps": 60,
        "step_size": 0.02,
    },
    use_measured_mutual_info=True,
)

characteristic matrix :
 [[1.00000000e+00 3.10862447e-15 3.10862447e-15 2.45715471e-04
  3.77475828e-15 2.22044605e-15 2.22044605e-15 3.10862447e-15]
 [3.10862447e-15 1.00000000e+00 1.61602099e-01 3.10862447e-15
  1.99840144e-15 2.22044605e-15 4.21884749e-15 2.88657986e-15]
 [3.10862447e-15 1.61602099e-01 1.00000000e+00 2.88657986e-15
  3.10862447e-15 2.66453526e-15 3.77475828e-15 4.44089210e-15]
 [2.45715471e-04 3.10862447e-15 2.88657986e-15 1.00000000e+00
  2.88657986e-15 3.77475828e-15 4.44089210e-15 3.10862447e-15]
 [3.77475828e-15 1.99840144e-15 3.10862447e-15 2.88657986e-15
  3.13465931e-01 3.77475828e-15 2.66453526e-15 1.11022302e-15]
 [2.22044605e-15 2.22044605e-15 2.66453526e-15 3.77475828e-15
  3.77475828e-15 3.69188385e-02 1.33226763e-15 3.10862447e-15]
 [2.22044605e-15 4.21884749e-15 3.77475828e-15 4.44089210e-15
  2.66453526e-15 1.33226763e-15 1.00000000e+00 6.73929407e-02]
 [3.10862447e-15 2.88657986e-15 4.44089210e-15 3.10862447e-15
  1.11022302e-15 3.10862447e-15 6.7392

In [40]:
covariance_matrix_inference(
    graph8_state_prep_node,
    graph8_mat,
    step_size=0.05,
    num_steps=30,
    verbose=False,
)

covariance matrix :
 [[ 1.00000000e+00 -1.94289029e-16 -8.32667268e-17  9.99951678e-01
  -2.20738870e-16 -1.11068947e-16 -8.32667268e-17  1.38777878e-16]
 [-1.94289029e-16  1.00000000e+00  6.66423972e-01 -1.11022302e-16
   1.95860170e-18 -1.11092269e-16  2.77555756e-17 -2.77555756e-17]
 [-8.32667268e-17  6.66423972e-01  1.00000000e+00 -1.66533454e-16
  -5.35525495e-17 -6.99665807e-20 -2.77555756e-17  8.32667268e-17]
 [ 9.99951678e-01 -1.11022302e-16 -1.66533454e-16  1.00000000e+00
  -1.11022302e-16 -5.55111512e-17 -8.32667268e-17  8.32667268e-17]
 [-2.20738870e-16  1.95860170e-18 -5.35525495e-17 -1.11022302e-16
   2.33836997e-02 -4.85722573e-17 -1.67186321e-16  1.52022897e-18]
 [-1.11068947e-16 -1.11092269e-16 -6.99665807e-20 -5.55111512e-17
  -4.85722573e-17  9.99999823e-01  5.55344734e-17  2.77322534e-17]
 [-8.32667268e-17  2.77555756e-17 -2.77555756e-17 -8.32667268e-17
  -1.67186321e-16  5.55344734e-17  1.00000000e+00 -9.99999999e-01]
 [ 1.38777878e-16 -2.77555756e-17  8.32667268e-1

## 8-Qubit Graph State with Frustrated CNOT arrangement

In [41]:
graph8_cyclic_mat = qnp.array(
    [
        [1, 1, 1, 0, 0, 0, 0, 0],
        [1, 1, 1, 0, 0, 0, 0, 0],
        [1, 1, 1, 0, 0, 0, 0, 0],
        [0, 0, 0, 1, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 0, 0, 0],
        [0, 0, 0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 0, 0, 1, 1],
        [0, 0, 0, 0, 0, 0, 1, 1],
    ]
)

graph8_cyclic_state_prep_node = qnetvo.PrepareNode(
    wires=[0, 1, 2, 3, 4, 5, 6, 7],
    ansatz_fn=qnetvo.graph_state_fn([[0, 1], [1, 2], [2, 0], [6, 7]]),
)

In [44]:
characteristic_matrix_inference(
    graph8_cyclic_state_prep_node,
    graph8_cyclic_mat,
    mi_opt_kwargs={"num_steps": 60, "step_size": 0.02},
)

characteristic matrix :
 [[1.00000000e+00 8.07513851e-08 2.88267330e-05 6.88338275e-15
  6.88338275e-15 7.10542736e-15 7.32747196e-15 7.10542736e-15]
 [8.07513851e-08 1.00000000e+00 1.55064740e-04 6.88338275e-15
  6.88338275e-15 7.10542736e-15 6.88338275e-15 7.32747196e-15]
 [2.88267330e-05 1.55064740e-04 1.00000000e+00 6.88338275e-15
  6.88338275e-15 6.88338275e-15 7.10542736e-15 7.10542736e-15]
 [6.88338275e-15 6.88338275e-15 6.88338275e-15 1.05943751e-01
  6.77236045e-15 6.88338275e-15 6.88338275e-15 7.10542736e-15]
 [6.88338275e-15 6.88338275e-15 6.88338275e-15 6.77236045e-15
  1.22791150e-09 6.88338275e-15 6.88338275e-15 6.88338275e-15]
 [7.10542736e-15 7.10542736e-15 6.88338275e-15 6.88338275e-15
  6.88338275e-15 5.23989771e-02 7.10542736e-15 7.10542736e-15]
 [7.32747196e-15 6.88338275e-15 7.10542736e-15 6.88338275e-15
  6.88338275e-15 7.10542736e-15 1.00000000e+00 9.99999993e-01]
 [7.10542736e-15 7.32747196e-15 7.10542736e-15 7.10542736e-15
  6.88338275e-15 7.10542736e-15 9.9999

In [45]:
characteristic_matrix_inference(
    graph8_cyclic_state_prep_node,
    graph8_cyclic_mat,
    mi_opt_kwargs={"num_steps": 60, "step_size": 0.02},
    use_measured_mutual_info=True,
)

characteristic matrix :
 [[1.00000000e+00 7.18145230e-05 4.07131922e-03 2.88657986e-15
  3.10862447e-15 3.55271368e-15 2.88657986e-15 2.66453526e-15]
 [7.18145230e-05 1.00000000e+00 1.04000346e-07 3.33066907e-15
  3.10862447e-15 3.77475828e-15 3.10862447e-15 2.88657986e-15]
 [4.07131922e-03 1.04000346e-07 1.00000000e+00 3.55271368e-15
  3.33066907e-15 2.44249065e-15 2.88657986e-15 2.66453526e-15]
 [2.88657986e-15 3.33066907e-15 3.55271368e-15 7.30200885e-04
  3.10862447e-15 1.99840144e-15 4.21884749e-15 4.88498131e-15]
 [3.10862447e-15 3.10862447e-15 3.33066907e-15 3.10862447e-15
  4.69566525e-01 3.10862447e-15 1.99840144e-15 2.66453526e-15]
 [3.55271368e-15 3.77475828e-15 2.44249065e-15 1.99840144e-15
  3.10862447e-15 6.67997304e-02 2.66453526e-15 3.77475828e-15]
 [2.88657986e-15 3.10862447e-15 2.88657986e-15 4.21884749e-15
  1.99840144e-15 2.66453526e-15 1.00000000e+00 9.94741824e-01]
 [2.66453526e-15 2.88657986e-15 2.66453526e-15 4.88498131e-15
  2.66453526e-15 3.77475828e-15 9.9474

In [46]:
covariance_matrix_inference(
    graph8_cyclic_state_prep_node,
    graph8_cyclic_mat,
    step_size=0.05,
    num_steps=30,
    verbose=False,
)

covariance matrix :
 [[ 1.00000000e+00 -2.24856182e-01  5.80274910e-01 -1.11022302e-16
   5.55111512e-17  0.00000000e+00 -8.32667268e-17  2.77555756e-17]
 [-2.24856182e-01  1.00000000e+00 -2.94178694e-01  5.54818204e-17
  -1.66625011e-16  2.76851536e-17  3.08148791e-33  8.32667268e-17]
 [ 5.80274910e-01 -2.94178694e-01  1.00000000e+00  2.93308513e-20
  -2.76640184e-17 -2.76851536e-17 -8.32667268e-17  2.77555756e-17]
 [-1.11022302e-16  5.54818204e-17  2.93308513e-20  1.05647611e-03
  -1.19262239e-16  1.67183975e-16 -1.10992972e-16 -5.54209329e-17]
 [ 5.55111512e-17 -1.66625011e-16 -2.76640184e-17 -1.19262239e-16
   9.99997280e-01 -3.64986497e-18  5.56027084e-17 -1.66167225e-16]
 [ 0.00000000e+00  2.76851536e-17 -2.76851536e-17  1.67183975e-16
  -3.64986497e-18  9.99998391e-01  1.66603876e-16 -1.10740614e-16]
 [-8.32667268e-17  3.08148791e-33 -8.32667268e-17 -1.10992972e-16
   5.56027084e-17  1.66603876e-16  1.00000000e+00 -1.00000000e+00]
 [ 2.77555756e-17  8.32667268e-17  2.77555756e-1

## 12-Qubit Graph State

In [47]:
graph12_mat = qnp.array(
    [
        [1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
        [0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1],
    ]
)
graph12_state_prep_node = qnetvo.PrepareNode(
    wires=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11],
    ansatz_fn=qnetvo.graph_state_fn([[0, 1], [1, 2], [6, 7], [4, 5], [4, 9], [4, 11], [11, 10]]),
)

In [48]:
characteristic_matrix_inference(
    graph12_state_prep_node, graph12_mat, mi_opt_kwargs={"num_steps": 60, "step_size": 0.02}
)

characteristic matrix :
 [[1.00000000e+00 1.83151931e-03 7.06604685e-03 9.99200722e-15
  9.99200722e-15 1.02140518e-14 9.76996262e-15 1.02140518e-14
  9.54791801e-15 9.76996262e-15 1.06581410e-14 1.08801856e-14]
 [1.83151931e-03 1.00000000e+00 1.64638564e-02 9.76996262e-15
  1.02140518e-14 1.02140518e-14 1.02140518e-14 9.76996262e-15
  9.76996262e-15 1.02140518e-14 1.06581410e-14 1.13242749e-14]
 [7.06604685e-03 1.64638564e-02 1.00000000e+00 9.76996262e-15
  9.99200722e-15 9.99200722e-15 9.76996262e-15 9.99200722e-15
  9.76996262e-15 1.02140518e-14 1.06581410e-14 1.11022302e-14]
 [9.99200722e-15 9.76996262e-15 9.76996262e-15 8.96491503e-01
  9.54791801e-15 9.76996262e-15 9.99200722e-15 1.02140518e-14
  9.99200722e-15 9.76996262e-15 1.06581410e-14 1.04360964e-14]
 [9.99200722e-15 1.02140518e-14 9.99200722e-15 9.54791801e-15
  1.00000000e+00 9.99917647e-01 9.76996262e-15 1.02140518e-14
  9.54791801e-15 9.78622604e-01 1.06581410e-14 1.11022302e-14]
 [1.02140518e-14 1.02140518e-14 9.992007

In [52]:
characteristic_matrix_inference(
    graph12_state_prep_node,
    graph12_mat,
    mi_opt_kwargs={"num_steps": 60, "step_size": 0.02},
    use_measured_mutual_info=True,
)

characteristic matrix :
 [[ 1.00000000e+00  1.70850332e-01  5.24042271e-02  3.10862447e-15
   4.44089210e-15  4.44089210e-15  4.44089210e-15  4.44089210e-15
   7.54951657e-15  1.77635684e-15  2.22044605e-14  1.73194792e-14]
 [ 1.70850332e-01  1.00000000e+00  8.45939026e-02  4.21884749e-15
   4.44089210e-15  3.10862447e-15  4.44089210e-15  5.77315973e-15
   3.10862447e-15  2.88657986e-15  8.88178420e-16  2.17603713e-14]
 [ 5.24042271e-02  8.45939026e-02  1.00000000e+00  4.44089210e-15
   4.44089210e-15  3.10862447e-15  3.10862447e-15  2.88657986e-15
   4.44089210e-15  1.77635684e-15  2.08721929e-14 -1.42108547e-14]
 [ 3.10862447e-15  4.21884749e-15  4.44089210e-15  1.51597539e-02
   4.21884749e-15  4.21884749e-15  4.66293670e-15  3.77475828e-15
   6.66133815e-16  1.33226763e-15  1.64313008e-14 -6.43929354e-15]
 [ 4.44089210e-15  4.44089210e-15  4.44089210e-15  4.21884749e-15
   1.00000000e+00  7.50452954e-01  4.66293670e-15  4.44089210e-15
   6.43929354e-15  4.17005473e-02  1.55431223e-

In [53]:
covariance_matrix_inference(
    graph12_state_prep_node,
    graph12_mat,
    num_steps=20,
    step_size=0.05,
    verbose=False,
)

covariance matrix :
 [[ 1.00000000e+00 -7.95319387e-01  8.57655404e-01 -2.23473252e-19
   1.66533454e-16  2.77555756e-17  2.77555756e-17 -5.55111512e-17
  -1.75685059e-16  2.77555756e-17  0.00000000e+00  1.05471187e-15]
 [-7.95319387e-01  1.00000000e+00 -8.63641516e-01  2.23473252e-19
  -1.38777878e-16 -5.55111512e-17  1.11022302e-16  0.00000000e+00
  -3.23915302e-16 -8.32667268e-17 -8.32667268e-17  6.66133815e-16]
 [ 8.57655404e-01 -8.63641516e-01  1.00000000e+00  2.74203657e-17
   1.38777878e-16  5.55111512e-17 -8.32667268e-17 -2.77555756e-17
   1.25050471e-16 -2.77555756e-17  0.00000000e+00  8.88178420e-16]
 [-2.23473252e-19  2.23473252e-19  2.74203657e-17  9.99995948e-01
  -1.11736626e-19  5.56228879e-17  2.77555756e-17 -2.77555756e-17
  -3.44261295e-16  1.38889615e-16  2.77555756e-17  8.89854469e-16]
 [ 1.66533454e-16 -1.38777878e-16  1.38777878e-16 -1.11736626e-19
   1.00000000e+00  8.26821620e-02 -8.32667268e-17  0.00000000e+00
  -4.57580247e-18  6.90736229e-02 -2.49800181e-16  

## A bunch of different entangled states

In [54]:
multi_ent_mat_full = qnp.zeros((8, 8))
multi_ent_mat = qnp.array(
    [
        [1, 1, 0, 0, 0, 0, 0],
        [1, 1, 0, 0, 0, 0, 0],
        [0, 0, 1, 1, 0, 0, 0],
        [0, 0, 1, 1, 0, 0, 0],
        [0, 0, 0, 0, 1, 1, 1],
        [0, 0, 0, 0, 1, 1, 1],
        [0, 0, 0, 0, 1, 1, 1],
    ]
)


def fn(settings, wires):
    qnetvo.shared_coin_flip_state([qnp.pi / 3], wires=[wires[0], wires[1], wires[7]])
    qnetvo.ghz_state([], wires=wires[2:4])
    qnetvo.W_state([], wires=wires[4:7])


multi_ent_prep_node = qnetvo.PrepareNode(
    wires=[0, 1, 2, 3, 4, 5, 6, 7],
    ansatz_fn=fn,
)

In [55]:
characteristic_matrix_inference(
    multi_ent_prep_node,
    multi_ent_mat_full,
    mi_opt_kwargs={
        "num_steps": 50,
        "step_size": 0.1,
    },
)

characteristic matrix :
 [[9.83916066e-01 8.11278124e-01 3.55271368e-15 3.77475828e-15
  3.77475828e-15 3.55271368e-15 3.99680289e-15 8.11266327e-01]
 [8.11278124e-01 9.51707902e-01 3.55271368e-15 3.77475828e-15
  3.77475828e-15 3.55271368e-15 3.99680289e-15 8.11266327e-01]
 [3.55271368e-15 3.55271368e-15 1.00000000e+00 9.99999999e-01
  3.77475828e-15 3.99680289e-15 3.77475828e-15 3.55271368e-15]
 [3.77475828e-15 3.77475828e-15 9.99999999e-01 1.00000000e+00
  3.77475828e-15 3.99680289e-15 3.77475828e-15 3.55271368e-15]
 [3.77475828e-15 3.77475828e-15 3.77475828e-15 3.77475828e-15
  9.99181425e-01 3.46466678e-01 3.33943277e-01 3.77475828e-15]
 [3.55271368e-15 3.55271368e-15 3.99680289e-15 3.99680289e-15
  3.46466678e-01 9.29144337e-01 3.32109251e-01 3.55271368e-15]
 [3.99680289e-15 3.99680289e-15 3.77475828e-15 3.77475828e-15
  3.33943277e-01 3.32109251e-01 9.30448460e-01 3.99680289e-15]
 [8.11266327e-01 8.11266327e-01 3.55271368e-15 3.55271368e-15
  3.77475828e-15 3.55271368e-15 3.9968

In [57]:
characteristic_matrix_inference(
    multi_ent_prep_node,
    multi_ent_mat_full,
    mi_opt_kwargs={
        "num_steps": 50,
        "step_size": 0.1,
    },
    use_measured_mutual_info=True,
)

characteristic matrix :
 [[9.59802863e-01 8.11569634e-03 1.55431223e-15 1.55431223e-15
  1.33226763e-15 1.33226763e-15 1.77635684e-15 6.33485473e-03]
 [8.11569634e-03 8.58852782e-01 1.11022302e-15 1.77635684e-15
  1.55431223e-15 2.44249065e-15 1.55431223e-15 6.22399526e-05]
 [1.55431223e-15 1.11022302e-15 1.00000000e+00 9.98787715e-01
  1.33226763e-15 1.99840144e-15 1.77635684e-15 6.66133815e-16]
 [1.55431223e-15 1.77635684e-15 9.98787715e-01 1.00000000e+00
  1.77635684e-15 1.77635684e-15 2.44249065e-15 1.33226763e-15]
 [1.33226763e-15 1.55431223e-15 1.33226763e-15 1.77635684e-15
  9.81626845e-01 3.47024754e-01 3.49967464e-01 2.44249065e-15]
 [1.33226763e-15 2.44249065e-15 1.99840144e-15 1.77635684e-15
  3.47024754e-01 9.80696615e-01 3.49216547e-01 1.33226763e-15]
 [1.77635684e-15 1.55431223e-15 1.77635684e-15 2.44249065e-15
  3.49967464e-01 3.49216547e-01 9.91767144e-01 2.44249065e-15]
 [6.33485473e-03 6.22399526e-05 6.66133815e-16 1.33226763e-15
  2.44249065e-15 1.33226763e-15 2.4424

In [56]:
covariance_matrix_inference(
    multi_ent_prep_node,
    multi_ent_mat,
    meas_wires=[0, 1, 2, 3, 4, 5, 6],
    num_steps=20,
    step_size=0.05,
    verbose=False,
)

covariance matrix :
 [[ 9.99993914e-01  1.98301183e-03  2.81664161e-17  2.74816820e-17
  -2.96356976e-17  5.73542949e-17 -7.86385388e-18]
 [ 1.98301183e-03  9.28209883e-01 -3.86462899e-17  2.57641933e-17
  -1.22518234e-16 -4.79217360e-17  1.30971622e-16]
 [ 2.81664161e-17 -3.86462899e-17  1.00000000e+00 -9.99999993e-01
  -4.44501684e-21  1.12076662e-16  2.67801137e-17]
 [ 2.74816820e-17  2.57641933e-17 -9.99999993e-01  1.00000000e+00
   2.77585390e-17  8.25638205e-17  8.39170348e-17]
 [-2.96356976e-17 -1.22518234e-16 -4.44501684e-21  2.77585390e-17
   9.99999999e-01  6.66546893e-01  6.66546069e-01]
 [ 5.73542949e-17 -4.79217360e-17  1.12076662e-16  8.25638205e-17
   6.66546893e-01  9.99959916e-01  6.66578150e-01]
 [-7.86385388e-18  1.30971622e-16  2.67801137e-17  8.39170348e-17
   6.66546069e-01  6.66578150e-01  9.99965690e-01]]
decision matrix :
 [[1 0 0 0 0 0 0]
 [0 1 0 0 0 0 0]
 [0 0 1 1 0 0 0]
 [0 0 1 1 0 0 0]
 [0 0 0 0 1 1 1]
 [0 0 0 0 1 1 1]
 [0 0 0 0 1 1 1]]
expected matrix :
 [

## Finite Shot Examples

In [58]:
graph5_mat = qnp.array(
    [
        [1, 1, 1, 0, 0],
        [1, 1, 1, 0, 0],
        [1, 1, 1, 0, 0],
        [0, 0, 0, 1, 1],
        [0, 0, 0, 1, 1],
    ]
)

qnode_kwargs = {
    "diff_method": "parameter-shift",
}

graph5_state_prep_node = qnetvo.PrepareNode(
    wires=[0, 1, 2, 3, 4],
    ansatz_fn=qnetvo.graph_state_fn([[0, 1], [1, 2], [2, 0], [3, 4]]),
)

## 1000 Shots

In [61]:
characteristic_matrix_inference(
    graph5_state_prep_node,
    graph5_mat,
    qnode_kwargs=qnode_kwargs,
    dev_kwargs={"shots": 1000},
    mi_opt_kwargs={
        "num_steps": 60,
        "step_size": 0.02,
    },
)

characteristic matrix :
 [[9.98958123e-01 5.09038418e-04 4.31651129e-04 1.00734747e-03
  1.00734747e-03]
 [5.09038418e-04 9.99927864e-01 3.80784938e-04 1.47526962e-06
  1.47526962e-06]
 [4.31651129e-04 3.80784938e-04 9.99711442e-01 2.20299521e-03
  2.20299521e-03]
 [1.00734747e-03 1.47526962e-06 2.20299521e-03 9.99927864e-01
  9.98727169e-01]
 [1.00734747e-03 1.47526962e-06 2.20299521e-03 9.98727169e-01
  9.99953833e-01]]
decision matrix :
 [[1 0 0 0 0]
 [0 1 0 0 0]
 [0 0 1 0 0]
 [0 0 0 1 1]
 [0 0 0 1 1]]
expected matrix :
 [[1 1 1 0 0]
 [1 1 1 0 0]
 [1 1 1 0 0]
 [0 0 0 1 1]
 [0 0 0 1 1]]
distance to expected :  2.448416462983034


In [63]:
covariance_matrix_inference(
    graph5_state_prep_node,
    graph5_mat,
    step_size=0.2,
    num_steps=6,
    dev_kwargs={"shots": 1000},
    qnode_kwargs=qnode_kwargs,
)

covariance matrix :
 [[ 0.999804 -0.999804 -0.999804 -0.025944 -0.025944]
 [-0.999804  0.999804  0.999804  0.025944  0.025944]
 [-0.999804  0.999804  0.999804  0.025944  0.025944]
 [-0.025944  0.025944  0.025944  0.999984  0.999984]
 [-0.025944  0.025944  0.025944  0.999984  0.999984]]
decision matrix :
 [[1 1 1 0 0]
 [1 1 1 0 0]
 [1 1 1 0 0]
 [0 0 0 1 1]
 [0 0 0 1 1]]
expected matrix :
 [[1 1 1 0 0]
 [1 1 1 0 0]
 [1 1 1 0 0]
 [0 0 0 1 1]
 [0 0 0 1 1]]


## 500 Shots

In [64]:
characteristic_matrix_inference(
    graph5_state_prep_node,
    graph5_mat,
    mi_opt_kwargs={
        "num_steps": 40,
        "step_size": 0.1,
    },
    dev_kwargs={"shots": 500},
    qnode_kwargs=qnode_kwargs,
)

characteristic matrix :
 [[0.99971144 0.99958446 0.99958446 0.00234693 0.00234693]
 [0.99958446 0.99334181 0.99958446 0.00234693 0.00234693]
 [0.99958446 0.99958446 0.99666191 0.00234693 0.00234693]
 [0.00234693 0.00234693 0.00234693 0.99884554 0.99884554]
 [0.00234693 0.00234693 0.00234693 0.99884554 0.99943439]]
decision matrix :
 [[1 1 1 0 0]
 [1 1 1 0 0]
 [1 1 1 0 0]
 [0 0 0 1 1]
 [0 0 0 1 1]]
expected matrix :
 [[1 1 1 0 0]
 [1 1 1 0 0]
 [1 1 1 0 0]
 [0 0 0 1 1]
 [0 0 0 1 1]]
distance to expected :  0.011269812189724933


In [66]:
covariance_matrix_inference(
    graph5_state_prep_node,
    graph5_mat,
    step_size=0.2,
    num_steps=10,
    dev_kwargs={"shots": 500},
    qnode_kwargs=qnode_kwargs,
)

covariance matrix :
 [[ 0.995904  0.983648 -0.995904 -0.050048  0.050048]
 [ 0.983648  0.995376 -0.983648 -0.054176  0.054176]
 [-0.995904 -0.983648  0.995904  0.050048 -0.050048]
 [-0.050048 -0.054176  0.050048  0.998976 -0.998976]
 [ 0.050048  0.054176 -0.050048 -0.998976  0.998976]]
decision matrix :
 [[1 1 1 0 0]
 [1 1 1 0 0]
 [1 1 1 0 0]
 [0 0 0 1 1]
 [0 0 0 1 1]]
expected matrix :
 [[1 1 1 0 0]
 [1 1 1 0 0]
 [1 1 1 0 0]
 [0 0 0 1 1]
 [0 0 0 1 1]]


## 100 Shots

In [67]:
characteristic_matrix_inference(
    graph5_state_prep_node,
    graph5_mat,
    mi_opt_kwargs={
        "num_steps": 40,
        "step_size": 0.1,
    },
    dev_kwargs={"shots": 100},
    qnode_kwargs=qnode_kwargs,
)

characteristic matrix :
 [[0.98581504 1.         1.         0.01048037 0.01048037]
 [1.         0.99971144 1.         0.01048037 0.01048037]
 [1.         1.         1.         0.01048037 0.01048037]
 [0.01048037 0.01048037 0.01048037 0.99537844 0.99537844]
 [0.01048037 0.01048037 0.01048037 0.99537844 0.99277445]]
decision matrix :
 [[1 1 1 0 0]
 [1 1 1 0 0]
 [1 1 1 0 0]
 [0 0 0 1 1]
 [0 0 0 1 1]]
expected matrix :
 [[1 1 1 0 0]
 [1 1 1 0 0]
 [1 1 1 0 0]
 [0 0 0 1 1]
 [0 0 0 1 1]]
distance to expected :  0.04044304888172123


In [69]:
covariance_matrix_inference(
    graph5_state_prep_node,
    graph5_mat,
    step_size=0.1,
    num_steps=40,
    dev_kwargs={"shots": 100},
    qnode_kwargs=qnode_kwargs,
)

covariance matrix :
 [[ 0.96   -0.96    0.96   -0.128  -0.128 ]
 [-0.96    0.96   -0.96    0.128   0.128 ]
 [ 0.96   -0.96    0.96   -0.128  -0.128 ]
 [-0.128   0.128  -0.128   0.9984  0.9984]
 [-0.128   0.128  -0.128   0.9984  0.9984]]
decision matrix :
 [[1 1 1 1 1]
 [1 1 1 1 1]
 [1 1 1 1 1]
 [1 1 1 1 1]
 [1 1 1 1 1]]
expected matrix :
 [[1 1 1 0 0]
 [1 1 1 0 0]
 [1 1 1 0 0]
 [0 0 0 1 1]
 [0 0 0 1 1]]


## 50 Shots

In [71]:
characteristic_matrix_inference(
    graph5_state_prep_node,
    graph5_mat,
    mi_opt_kwargs={
        "num_steps": 40,
        "step_size": 0.1,
    },
    dev_kwargs={"shots": 50},
    qnode_kwargs=qnode_kwargs,
)

characteristic matrix :
 [[0.9814539  0.98958752 0.98958752 0.02809002 0.02809002]
 [0.98958752 0.98958752 0.98958752 0.02809002 0.02809002]
 [0.98958752 0.98958752 0.99884554 0.02809002 0.02809002]
 [0.02809002 0.02809002 0.02809002 0.9814539  0.99884554]
 [0.02809002 0.02809002 0.02809002 0.99884554 0.9814539 ]]
decision matrix :
 [[1 1 1 0 0]
 [1 1 1 0 0]
 [1 1 1 0 0]
 [0 0 0 1 1]
 [0 0 0 1 1]]
expected matrix :
 [[1 1 1 0 0]
 [1 1 1 0 0]
 [1 1 1 0 0]
 [0 0 0 1 1]
 [0 0 0 1 1]]
distance to expected :  0.10612916066379295


In [72]:
covariance_matrix_inference(
    graph5_state_prep_node,
    graph5_mat,
    step_size=0.1,
    num_steps=40,
    dev_kwargs={"shots": 50},
    qnode_kwargs=qnode_kwargs,
)

covariance matrix :
 [[ 0.9744  0.0144  0.0144  0.0144  0.0144]
 [ 0.0144  0.9744  0.9744 -0.1456 -0.1456]
 [ 0.0144  0.9744  0.9744 -0.1456 -0.1456]
 [ 0.0144 -0.1456 -0.1456  0.9744  0.9744]
 [ 0.0144 -0.1456 -0.1456  0.9744  0.9744]]
decision matrix :
 [[1 0 0 0 0]
 [0 1 1 1 1]
 [0 1 1 1 1]
 [0 1 1 1 1]
 [0 1 1 1 1]]
expected matrix :
 [[1 1 1 0 0]
 [1 1 1 0 0]
 [1 1 1 0 0]
 [0 0 0 1 1]
 [0 0 0 1 1]]


## 10 Shots

In [73]:
characteristic_matrix_inference(
    graph5_state_prep_node,
    graph5_mat,
    mi_opt_kwargs={
        "num_steps": 40,
        "step_size": 0.1,
    },
    dev_kwargs={"shots": 10},
    qnode_kwargs=qnode_kwargs,
)

characteristic matrix :
 [[0.8812909  0.12451125 0.12451125 0.03485155 0.12451125]
 [0.12451125 0.97095059 0.04643934 0.00580215 0.01997309]
 [0.12451125 0.04643934 0.97095059 0.00580215 0.01997309]
 [0.03485155 0.00580215 0.00580215 0.97095059 0.55677965]
 [0.12451125 0.01997309 0.01997309 0.55677965 0.8812909 ]]
decision matrix :
 [[1 1 1 0 1]
 [1 1 0 0 0]
 [1 0 1 0 0]
 [0 0 0 1 1]
 [1 0 0 1 1]]
expected matrix :
 [[1 1 1 0 0]
 [1 1 1 0 0]
 [1 1 1 0 0]
 [0 0 0 1 1]
 [0 0 0 1 1]]
distance to expected :  2.3115465903198498


In [74]:
covariance_matrix_inference(
    graph5_state_prep_node,
    graph5_mat,
    step_size=0.1,
    num_steps=40,
    dev_kwargs={"shots": 10},
    qnode_kwargs=qnode_kwargs,
)

covariance matrix :
 [[ 1.   -1.   -0.4  -0.6  -0.6 ]
 [-1.    1.    0.4   0.6   0.6 ]
 [-0.4   0.4   0.96  0.8   0.8 ]
 [-0.6   0.6   0.8   1.    1.  ]
 [-0.6   0.6   0.8   1.    1.  ]]
decision matrix :
 [[1 1 1 1 1]
 [1 1 1 1 1]
 [1 1 1 1 1]
 [1 1 1 1 1]
 [1 1 1 1 1]]
expected matrix :
 [[1 1 1 0 0]
 [1 1 1 0 0]
 [1 1 1 0 0]
 [0 0 0 1 1]
 [0 0 0 1 1]]


## 5 Shots


In [75]:
characteristic_matrix_inference(
    graph5_state_prep_node,
    graph5_mat,
    mi_opt_kwargs={
        "num_steps": 40,
        "step_size": 0.1,
    },
    dev_kwargs={"shots": 5},
    qnode_kwargs=qnode_kwargs,
)

characteristic matrix :
 [[0.97095059 0.41997309 0.32192809 0.01997309 0.01997309]
 [0.41997309 0.97095059 0.17095059 0.01997309 0.01997309]
 [0.32192809 0.17095059 0.97095059 0.32192809 0.32192809]
 [0.01997309 0.01997309 0.32192809 0.72192809 0.97095059]
 [0.01997309 0.01997309 0.32192809 0.97095059 0.97095059]]
decision matrix :
 [[1 1 1 0 0]
 [1 1 1 0 0]
 [1 1 1 1 1]
 [0 0 1 1 1]
 [0 0 1 1 1]]
expected matrix :
 [[1 1 1 0 0]
 [1 1 1 0 0]
 [1 1 1 0 0]
 [0 0 0 1 1]
 [0 0 0 1 1]]
distance to expected :  1.8620420658973


In [76]:
covariance_matrix_inference(
    graph5_state_prep_node,
    graph5_mat,
    step_size=0.1,
    num_steps=10,
    dev_kwargs={"shots": 5},
    qnode_kwargs=qnode_kwargs,
)

covariance matrix :
 [[ 0.96 -0.64  0.32 -0.48 -0.16]
 [-0.64  0.96  0.32  0.32 -0.16]
 [ 0.32  0.32  0.64 -0.16 -0.32]
 [-0.48  0.32 -0.16  0.64  0.48]
 [-0.16 -0.16 -0.32  0.48  0.96]]
decision matrix :
 [[1 1 1 1 1]
 [1 1 1 1 1]
 [1 1 1 1 1]
 [1 1 1 1 1]
 [1 1 1 1 1]]
expected matrix :
 [[1 1 1 0 0]
 [1 1 1 0 0]
 [1 1 1 0 0]
 [0 0 0 1 1]
 [0 0 0 1 1]]
